# Neurorobotics with PyNN and PyBullet

In this Notebook we will build and execute a neuro-robotics experiment. Für the neural simulation we will use [PyNN](https://neuralensemble.org/PyNN/) and [Nest](https://www.nest-initiative.org/?page=Software). For the simulation of the physic we use [PyBullet](https://github.com/bulletphysics/bullet3/tree/master/examples/pybullet). 

This experiment is a re-creation of the [braitenberg-husky-experiment](https://bitbucket.org/hbpneurorobotics/experiments/src/development/braitenberg_husky/) in the neurorobotics plattform of the human brain project. 

We start by importing all modules we will use in this notebook. Make sure to install only python**3**-versions of all dependencies. 

You can not just install these via pip:

- _pybullet_ needs to be installed from source to get numpy-support. Otherwise it will be slower, and getCameraImage doesn't return a numpy-array. 
- _nest_ can only be installed from source. We must use version 2.16, because the current master is not yet compatible with _pynn_. Also _nest_ is compiled with _libnreuosim_, which needs a workaround until [the PR](https://github.com/nest/nest-simulator/pull/1235) is merged. I'm not 100% certain if _libneurosim_ is even required for this project, but _nest_ gives a warning if it's missing, so we will install it. 
- there is a warning "UserWarning: Unable to install NEST extensions. Certain models may not be available", which doesn't seem to affect this project. Please ignore it. 

In [1]:
import time
import numpy as np
import csv
import random

import subprocess
import re

from matplotlib import pyplot as plt
from matplotlib import animation
from pyNN.utility.plotting import Figure, Panel
from quantities import mV

In [2]:
def isFinished(processes):
    
    for process in processes:
        if(process.poll() == None):
            return False
    
    return True

In [3]:
output_min = []
error_min =200

print("Starting")

for N in range(5):
    
    # Start time measurement
    start = time.time()
    
    weight_list = []
    processes = []

    # Execute all subprocesses and save outputs into files
    for i in range(40):
        
        # Generate Random weights
        WEIGHT_RED_TO_ACTOR = 10**random.uniform(-4, -2)
        WEIGHT_RED_TO_GO_ON = 10**random.uniform(-4, -2)
        WEIGHT_GREEN_BLUE_TO_ACTOR = 10**random.uniform(-4, -2)
        WEIGHT_GO_ON_TO_RIGHT_ACTOR = 10**random.uniform(-4, -2)

        filename = "output/log" + str(i) + ".txt"
        with open(filename, "w") as file:
            sp = subprocess.Popen(['python', 'OptimizationJob.py', str(WEIGHT_RED_TO_ACTOR), str(WEIGHT_RED_TO_GO_ON), str(WEIGHT_GREEN_BLUE_TO_ACTOR), str(WEIGHT_GO_ON_TO_RIGHT_ACTOR)], stdout=file)
            processes.append( sp ) 
            
        weight_list.append([WEIGHT_RED_TO_ACTOR, WEIGHT_RED_TO_GO_ON, WEIGHT_GREEN_BLUE_TO_ACTOR, WEIGHT_GO_ON_TO_RIGHT_ACTOR])
            
    # Wait until all processes are finished
    while isFinished(processes) == False:
        time.sleep(0.5)
    
    # Read outputs
    for i in range(40):

        with open("output/log" + str(i) + ".txt") as file:
            filetext = file.read()

            # Find fitness value through regex
            matches = re.findall("(.*[^=])=(.*)", filetext)
            error = float(matches[0][1])

            if error < error_min:
                error_min = error
                print(weight_list[i])
                print(error)

    # Time measurement
    end = time.time()
    
    # Ausgabe
    print('Time taken in seconds -', end - start)
    
    
print("Finished")

Starting
[0.009355252427922334, 0.0016593673536640702, 0.0001951361264199021, 0.0009955911862304602]
0.5826026370208032
[0.0013704662089159291, 0.0003961323051802113, 0.00423946771208724, 0.00010115625056028498]
0.4293368323765484
[0.0033920476623201983, 0.0008606399452036793, 0.0038706569847886405, 0.002544893908646982]
0.27471606897009226
Time taken in seconds - 73.84484672546387
Time taken in seconds - 74.35172033309937
[0.008531327064261574, 0.0007676469457076234, 0.007815988697869166, 0.005686854884066715]
0.24418609397952454
Time taken in seconds - 73.86934566497803
Time taken in seconds - 73.21843028068542
Time taken in seconds - 73.86296772956848
Finished


### Time measurement

| Number of subprocesses   |  Elapsed Time   |  Elapsed Time per subprocess  |
|--------------------------|-----------------|-------------------------------|
| 1                        |    39.06 s      |         39.06 s               |
| 5                        |    42.63 s      |          8.52 s               |
| 10                       |    44.72 s      |          4.47 s               |
| 20                       |    49.58 s      |          2.48 s               |
| 30                       |    66.34 s      |          2.21 s               |
| 40                       |    73.10 s      |          1.83 s               |
| 50                       |    92.97 s      |          1.86 s               |
| 80                       |   149.80 s      |          1.87 s               |
|100                       |   188.15 s      |          1.88 s               |
|200                       |   382.65 s      |          1.91 s               |
|300                       |   592.33 s      |          1.97 s               |
|500                       |   993.66 s      |          1.99 s               |
|800                       |  1599.73 s      |          1.99 s               |